In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os


def random_init(size):
    in_dim = size[0]
    stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=stddev)

In [2]:
X = tf.placeholder(tf.float32, shape=[None, 784])

d_weights1 = tf.Variable(random_init([784, 128]))
d_biases1 = tf.Variable(tf.zeros(shape=[128]))

d_weights2 = tf.Variable(random_init([128, 1]))
d_biases2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [d_weights1, d_weights2, d_biases1, d_biases2]


Z = tf.placeholder(tf.float32, shape=[None, 100])

g_weights1 = tf.Variable(random_init([100, 392]))
g_biases1 = tf.Variable(tf.zeros(shape=[392]))

# g_weights2 = tf.Variable(random_init([128, 128]))
# g_biases2 = tf.Variable(tf.zeros(shape=[128]))

g_weights3 = tf.Variable(random_init([392, 784]))
g_biases3 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [g_weights1, g_weights3, g_biases1, g_biases3]

In [3]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [4]:
def generator(z):
    g_layer1 = tf.nn.relu(tf.matmul(z, g_weights1) + g_biases1)
#     g_layer2 = tf.nn.relu(tf.matmul(g_layer1, g_weights2) + g_biases2)
    g_log_prob = tf.matmul(g_layer1, g_weights3) + g_biases3
    g_prob = tf.nn.sigmoid(g_log_prob)

    return g_prob

In [5]:
def discriminator(x):
    d_layer1 = tf.nn.relu(tf.matmul(x, d_weights1) + d_biases1)
    d_logit = tf.matmul(d_layer1, d_weights2) + d_biases2
    d_prob = tf.nn.sigmoid(d_logit)

    return d_prob, d_logit

In [6]:
# plotting function

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [7]:
g_sample = generator(Z)
d_real, d_logit_real = discriminator(X)
d_fake, d_logit_fake = discriminator(g_sample)

d_loss = -tf.reduce_mean(tf.log(d_real) + tf.log(1. - d_fake))
g_loss = -tf.reduce_mean(tf.log(d_fake))

In [8]:
d_solver = tf.train.AdamOptimizer().minimize(d_loss, var_list=theta_D)
g_solver = tf.train.AdamOptimizer().minimize(g_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(20001):
    if it % 1000 == 0:
        samples = sess.run(g_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, d_loss_curr = sess.run([d_solver, d_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, g_loss_curr = sess.run([g_solver, g_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(d_loss_curr))
        print('G_loss: {:.4}'.format(g_loss_curr))
        print()


Iter: 0
D loss: 1.515
G_loss: 2.554

Iter: 1000
D loss: 0.0188
G_loss: 7.052

Iter: 2000
D loss: 0.029
G_loss: 6.152

Iter: 3000
D loss: 0.03708
G_loss: 6.184

Iter: 4000
D loss: 0.1047
G_loss: 6.041

Iter: 5000
D loss: 0.1883
G_loss: 4.216

Iter: 6000
D loss: 0.3129
G_loss: 3.658

Iter: 7000
D loss: 0.3847
G_loss: 3.283

Iter: 8000
D loss: 0.4023
G_loss: 3.132

Iter: 9000
D loss: 0.8032
G_loss: 3.189

Iter: 10000
D loss: 0.6348
G_loss: 2.589

Iter: 11000
D loss: 0.656
G_loss: 3.349

Iter: 12000
D loss: 0.6913
G_loss: 2.66

Iter: 13000
D loss: 0.6043
G_loss: 2.439

Iter: 14000
D loss: 0.9143
G_loss: 2.4

Iter: 15000
D loss: 0.7275
G_loss: 2.405

Iter: 16000
D loss: 0.8048
G_loss: 1.942

Iter: 17000
D loss: 0.721
G_loss: 2.089

Iter: 18000
D loss: 0.8571
G_loss: 2.144

Iter: 19000
D loss: 0.7563
G_loss: 2.247

Iter: 20000
D loss: 0.7718
G_loss: 1.635



In [10]:
[2.596, 6.897, 6.356, 5.928, 4.588, 4.099, 3.724]

[2.596, 6.897, 6.356, 5.928, 4.588, 4.099, 3.724]